In [1]:
%load_ext cudf.pandas
from pathlib import Path
from sklearn_api_model import save_object, Model
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
# from ..forecasting_models.tools import save_object

/usr/lib/python3.10/importlib/__init__.py:126: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  return _bootstrap._gcd_import(name[level:], package, level)
/home/maxime/Documents/WORKSPACES/forecasting_models/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
reg = xgb.XGBRegressor()
reg.set_params(device='cuda')
model = Model(model=reg, loss='rmse', name='Model')

In [ ]:
features = pd.read_csv("/home/maxime/Documents/WORKSPACES/ARS_Project/data/dataframes/features_encoded/features_CHU Dijon.csv")
features.set_index('date_entree', inplace=True)

In [ ]:
features = features.dropna()

In [ ]:
train_set, test_set = train_test_split(features, test_size=0.2, random_state=42)

In [ ]:
X_train = train_set.drop("Total", axis=1)
y_train = train_set["Total"]

In [ ]:
X_test = test_set.drop("Total", axis=1)
y_test = test_set["Total"]

In [ ]:
param_grid = {
    'max_depth': [3, 4, 5, 6]
}

In [ ]:
fit_params = {
    }
model.fit(X_train, y_train, optimization='grid', grid_params=param_grid, fit_params=fit_params)

save_object(model, 'model.pkl', Path('data/models/'))

In [ ]:
print(model.predict(X_test))
print(model.score(X_test, y_test))